In [2]:
import pandas as pd
import numpy as np

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import pylab as plt

In [4]:
def clean_data():
    
    global db1
    db1 = db1[(db1.num_infections > 0)].reset_index()
    db1 = db1.drop(columns = ['index'], axis = 1)
    
    db1['date'] = pd.to_datetime(db1.date)
    
   
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad Valenciana' if x == 'Valenciana, Comunidad' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad de Madrid' if x == 'Madrid, Comunidad de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Región de de Murcia' if x == 'Murcia, Región de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Comunidad Foral de Navarra' if x == 'Navarra, Comunidad Foral de' else x)
    db1['autonomous_region'] = db1.autonomous_region.apply(lambda x: 'Principado de Asturias' if x == 'Asturias, Principado de' else x)
    
    
    db1['province'] = db1.province.apply(lambda x: 'Alicante' if x == 'Alicante/Alacant' else x)
    db1['province'] = db1.province.apply(lambda x: 'Castellón' if x == 'Castellón/Castelló' else x)
    db1['province'] = db1.province.apply(lambda x: 'Araba' if x == 'Araba/Álava' else x)
    
    db1['sex'] = db1.sex.apply(lambda x: 'UKNOWN' if x == 'NC' else x)
    
    db1['age_interval'] = db1.age_interval.apply(lambda x: 'UKNOWN' if x == 'NC' else x)

In [5]:
def cumulative(dataframe):
    
    cumu_num_infections = dataframe.num_infections.cumsum()
    cumu_num_hosp = dataframe.num_hosp.cumsum()
    cumu_num_uci = dataframe.num_uci.cumsum()
    cumu_num_dead = dataframe.num_dead.cumsum()

    cumulative = pd.DataFrame({'cumu_num_infections': cumu_num_infections, 'cumu_num_hosp': cumu_num_hosp, 
                               'cumu_num_uci': cumu_num_uci, 'cumu_num_dead': cumu_num_dead})
    dataframe = pd.concat([dataframe, cumulative], axis = 1)
    return dataframe

In [6]:
def freq_rel(dataframe):
    
    dataframe['Number of infection (%)'] = [round(i/ dataframe['num_infections'].sum(),3)*100 for i in dataframe['num_infections']]
    dataframe['Number of hospitalisation (%)'] = [round(i/ dataframe['num_hosp'].sum(),3)*100 for i in dataframe['num_hosp']]
    dataframe['Number of Intensive Care Unit (%)'] = [round(i/ dataframe['num_uci'].sum(),3)*100 for i in dataframe['num_uci']]
    dataframe['Number of deaths (%)'] = [round(i/ dataframe['num_dead'].sum(),3)*100 for i in dataframe['num_dead']]

In [7]:
def organise(column):
    
    name = db1.groupby(column).sum()
    freq_rel(name)
    
    return name

In [8]:
def mov_7_ave(dataframe):
    dataframe['ave_7_num_infections'] = dataframe.iloc[:,1].rolling(window=7).mean()
    dataframe['ave_7_num_hosp'] = dataframe.iloc[:,2].rolling(window=7).mean()
    dataframe['ave_7_num_uci'] = dataframe.iloc[:,3].rolling(window=7).mean()
    dataframe['ave_7_num_dead'] = dataframe.iloc[:,4].rolling(window=7).mean()
    
    return dataframe

In [9]:
db1 =pd.read_csv('/content/covid-19-sample.csv') 

In [10]:
db1

,date,autonomous_region,province,sex,age_interval,num_infections,num_hosp,num_uci,num_dead
0,2020-01-01,"Valenciana, Comunidad",Alicante/Alacant,H,0-9,0,0,0,0
1,2020-01-01,"Valenciana, Comunidad",Alicante/Alacant,H,10-19,0,0,0,0
2,2020-01-01,"Valenciana, Comunidad",Alicante/Alacant,H,20-29,0,0,0,0
3,2020-01-01,"Valenciana, Comunidad",Alicante/Alacant,H,30-39,0,0,0,0
4,2020-01-01,"Valenciana, Comunidad",Alicante/Alacant,H,40-49,0,0,0,0
...,...,...,...,...,...,...,...,...,...
822115,2021-06-10,Castilla y León,Zamora,NC,50-59,0,0,0,0
822116,2021-06-10,Castilla y León,Zamora,NC,60-69,0,0,0,0
822117,2021-06-10,Castilla y León,Zamora,NC,70-79,0,0,0,0
822118,2021-06-10,Castilla y León,Zamora,NC,80+,0,0,0,0


In [11]:
clean_data()

In [12]:
bydate = db1.groupby('date').sum().reset_index()
bydate = cumulative(bydate)
bydate = mov_7_ave(bydate)

In [13]:
bydate.shape

(480, 13)

# ARMA

In [14]:
from statsmodels.tsa.arima_model import ARMA

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [15]:
from sklearn.metrics import mean_squared_error as mse

In [16]:
infections = bydate[['date', 'ave_7_num_infections']].set_index('date')

In [17]:
infections = infections[6:]

In [18]:
infections

,ave_7_num_infections
date,
2020-02-23,1.428571
2020-02-24,1.571429
2020-02-25,1.571429
2020-02-26,2.428571
2020-02-27,5.285714
...,...
2021-06-06,4015.142857
2021-06-07,3911.142857
2021-06-08,3761.857143


# 5 days

In [19]:
train, test = infections[:-5], infections[-5:]

In [ ]:
res=[]
for p in range(1, 20):
  for q in range(1, 20):
    try:
      model=ARMA(train, order = (p, q)).fit(disp=False)
      res.append((p, q, mse(model.predict(len(train), len(infections)-1), test)**0.5))
    except:
      continue

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packa

In [ ]:
error = [e for p, q, e in res]
best = [res[i] for i, e in enumerate(error) if e == min(error)]

best

[(19, 11, 79.8133413349588)]

In [ ]:
res

[(1, 1, 336.7311769218717),
 (1, 2, 376.83325753064105),
 (1, 3, 332.28192933862033),
 (1, 4, 315.20849292308736),
 (1, 5, 338.8424755801726),
 (1, 6, 296.96862460522345),
 (1, 7, 261.89707521734346),
 (1, 9, 189.75519400094257),
 (1, 10, 203.10264061279392),
 (1, 11, 261.43034932111624),
 (1, 14, 292.396572721566),
 (1, 16, 234.35028193843348),
 (2, 1, 315.2060669230343),
 (2, 2, 356.4297159770225),
 (2, 3, 307.0895047710416),
 (2, 4, 281.0430029463828),
 (2, 5, 276.8716836804102),
 (2, 6, 280.4449436169506),
 (2, 7, 263.5867551490161),
 (2, 8, 314.9975942983921),
 (2, 9, 318.55230889373485),
 (2, 10, 302.6402073937875),
 (2, 11, 297.2317892132639),
 (2, 12, 288.54802977584774),
 (2, 13, 251.72374342556583),
 (2, 14, 252.70401478563534),
 (2, 15, 183.73171337747596),
 (2, 16, 218.23631126776084),
 (2, 17, 218.4352873057458),
 (2, 18, 216.61562561987205),
 (2, 19, 190.67397965108827),
 (3, 1, 341.20242315265085),
 (3, 2, 341.2916775799825),
 (3, 3, 283.156269111217),
 (3, 4, 314.767589

In [ ]:
res2=[]
for p in range(17, 23):
  for q in range(5, 20):
    try:
      model=ARMA(train, order = (p, q)).fit(disp=False)
      res2.append((p, q, mse(model.predict(len(train), len(infections)-1), test)**0.5))
    except:
      continue

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
/usr/local/lib/python3.7/dist-packages/statsmodels/base/model.py:512: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/usr/local/lib/

In [ ]:
error2 = [e for p, q, e in res2]
best2 = [res2[i] for i, e in enumerate(error2) if e == min(error2)]

best2